In [0]:
# practicing book recommender model



In [0]:
# imports


import numpy as pd
import pandas as pd
import pickle


from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import re

In [0]:
# load data sets
# from kaggle
# books.csv has 10k books
# ratings.csv ranging from 1-5 stars



In [0]:
books = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv')

In [0]:
ratings = pd.read_csv('https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv')

In [8]:
print(ratings.shape)
ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [9]:
print(books.shape)
books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [11]:
# trim down
# only need book id and title

cols = ['book_id', 'title']

books = books[cols]
books.head()

,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,"Twilight (Twilight, #1)"
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


In [0]:
# clean up titles
# there are empty spaces !!

def clean_book_titles(title):
  title = re.sub(r'\([^)]*\)', '', title) # handles brackets
  title = re.sub(' + ', ' ', title) #compresses multi spaces into a single space
  title = title.strip() # handles special characters
  return title

In [13]:
books['title'] = books['title'].apply(clean_book_titles)

books.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,book_id,title
0,1,The Hunger Games
1,2,Harry Potter and the Sorcerer's Stone
2,3,Twilight
3,4,To Kill a Mockingbird
4,5,The Great Gatsby


In [0]:
# so neat!


In [15]:
# feature matrix
# combine datasets

books_ratings = pd.merge(rating, books, on='book_id')

print(books_ratings.shape)

books_ratings.head()


(5976479, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


In [18]:
# remove dups

user_ratings = books_ratings.drop_duplicates(['user_id', 'title'])

print(user_ratings.shape)
user_ratings.head()

(5972713, 4)


,user_id,book_id,rating,title
0,1,258,5,The Shadow of the Wind
1,11,258,3,The Shadow of the Wind
2,143,258,4,The Shadow of the Wind
3,242,258,5,The Shadow of the Wind
4,325,258,4,The Shadow of the Wind


In [19]:
# create a matrix using user_rating

user_matrix = user_ratings.pivot(index='title', columns='user_id', values='rating').fillna(0)

user_matrix.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,53385,53386,53387,53388,53389,53390,53391,53392,53393,53394,53395,53396,53397,53398,53399,53400,53401,53402,53403,53404,53405,53406,53407,53408,53409,53410,53411,53412,53413,53414,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
#GIRLBOSS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
'Tis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1,000 Places to See Before You Die",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# compress it
# too large

compressed = csr_matrix(user_matrix.values)

In [21]:
# build and train knn

# unsupervised learning
# using cosine to measure space/distance

knn = NearestNeighbors(algorithm='brute', metric='cosine')

knn.fit(compressed)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [24]:
def get_recommendations(book_title, matrix=user_matrix, model=knn, topn=5):
    book_index = list(matrix.index).index(book_title)
    distances, indices = model.kneighbors(matrix.iloc[book_index,:].values.reshape(1,-1), n_neighbors=topn+1)
    print('Recommendations for {}:'.format(matrix.index[book_index]))
    for i in range(1, len(distances.flatten())):
        print('{}. {}, distance = {}'.format(i, matrix.index[indices.flatten()[i]], "%.3f"%distances.flatten()[i]))
    print()
    
get_recommendations("Harry Potter and the Sorcerer's Stone")
get_recommendations("Moby-Dick or, The Whale")
get_recommendations("Little Women")
get_recommendations("Charlie and the Chocolate Factory")

Recommendations for Harry Potter and the Sorcerer's Stone:
1. Harry Potter and the Prisoner of Azkaban, distance = 0.320
2. Harry Potter and the Chamber of Secrets, distance = 0.327
3. Harry Potter and the Goblet of Fire, distance = 0.331
4. Harry Potter and the Order of the Phoenix, distance = 0.342
5. Harry Potter and the Half-Blood Prince, distance = 0.348

Recommendations for Moby-Dick or, The Whale:
1. The Odyssey, distance = 0.701
2. A Tale of Two Cities, distance = 0.703
3. The Adventures of Huckleberry Finn, distance = 0.703
4. Frankenstein, distance = 0.710
5. The Old Man and the Sea, distance = 0.729

Recommendations for Little Women:
1. Pride and Prejudice, distance = 0.519
2. Jane Eyre, distance = 0.539
3. To Kill a Mockingbird, distance = 0.561
4. The Diary of a Young Girl, distance = 0.561
5. Sense and Sensibility, distance = 0.561

Recommendations for Charlie and the Chocolate Factory:
1. James and the Giant Peach, distance = 0.541
2. Matilda, distance = 0.543
3. The Wit